- 340604(화) 중앙대학교 군 장병 AISW 역량강화: 고급자연어처리 실습 자료입니다.
- 본 내용은 IIPL (Intelligent Information Processing Lab) 소속 석사과정 김영화 조교가 작성하였습니다.

---
## WEEK 01
- ChatGPT API Usage

### Setting

In [ ]:
!pip install datasets
!pip install transformers

### ChatGPTAPI


- openai 라이브러리 설치

In [ ]:
!pip install openai

In [ ]:
import openai
from datasets import load_dataset
from datasets import Dataset
import numpy as np
import json
from tqdm.auto import tqdm
import time

- https://platform.openai.com/account/api-keys 접속 후 로그인하여 API 키 발급 받기
  - 발급받은 API 키 복사해서 저장
  - 잃어버릴 경우 다시 생성해야 함

In [ ]:
MY_API_KEY = ''
openai.api_key = MY_API_KEY

### Dataset load
- NLI task는 두 문장에 대한 관계를 추론하는 문제로, 첫 번째 문장(promise)이 어떤 상황에 대한 정보를 제공함. "전제(premise)"라고 불리며 어떤 상황에 대한 정보를 제공
- 두 번째 문장(hypothesis)은, 전제에 대한 가정이나 주장을 나타냄.
- NLI 모델은 hypothesis가 premise를 "모순되지 않게(support)", "모순되게(contradict)", 또는 "중립적(neutral)"으로 표현핮는지를 판단하는 것이 목표
- 즉, label 3개 중 하나를 고르는 문제

In [ ]:
klue_nli = load_dataset('klue', 'nli')
print("Dataset loaded!")
train_data = klue_nli['train']
test_data = klue_nli['validation'][:300]
test_data = Dataset.from_dict(test_data)

In [ ]:
model = 'gpt-3.5-turbo'

In [ ]:
inference_result = []
for step, sample in tqdm(enumerate(test_data), total=len(test_data)):
    premise = sample['premise']
    hypothesis = sample['hypothesis']

    query = f"premise: {premise} hypothesis: {hypothesis} label:"
    messages = [
        {'role': 'system', 'content': 'only "entailment", "neutral", "contradiction" are allowed.'},
        {'role': 'user', 'content': query}
    ]
    while True:
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages
            )
            break
        except openai.error.RateLimitError as e:
            print(f"Rate limit exceeded. Retrying in 20 seconds. Error: {e}")
            time.sleep(20)

    pred = response['choices'][0]['message']['content'].lower()
    if 'entailment' in pred:
        pred = 0
    elif 'neutral' in pred:
        pred = 1
    elif 'contradiction' in pred:
        pred = 2
    else:
        print('error : out of label')
        print(step, pred)
        break
    inference_result.append(pred)


json_arr = json.dumps(inference_result)
with open('chatgpt.json', 'w') as f:
    f.write(json_arr)

## Problem
https://huggingface.co/models 에서 하나의 모델을 고르거나, 본인이 직접 한국어 NLI 모델을 학습하여 KLUE NLI 데이터셋 validation set에 대해 inference를 수행